In [18]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader
import math

In [19]:
proccessedData = torch.load("../dataset/Data/preproccessedData.pt")
proccessedDataMASK = torch.load("../dataset/Data/preproccessedDataMASKS.pt")

In [20]:
torch.triu(torch.ones(10,15),diagonal=0).bool()

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True,
          True,  True,  True,  True,  True],
        

In [21]:
pe = torch.zeros(4,10)
print(pe)
position = torch.arange(0,4,dtype=torch.float).unsqueeze(1)
print(position)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0.],
        [1.],
        [2.],
        [3.]])


In [22]:
class Blocks(nn.Module):
    def __init__(self,d_model,n_head):
        super().__init__()
        headsize = d_model // n_head
        self.sa_head = nn.MultiheadAttention(embed_dim=d_model,num_heads=n_head,batch_first=True)

        self.ffn = nn.Sequential(
            nn.Linear(d_model,4 * d_model),
            nn.ReLU(),
            nn.Linear(d_model * 4,d_model)
        )

        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)


    def forward(self,x,mask=None,padding=None):
        #output,weight (i ignored weights)
        att_out, _ = self.sa_head(self.ln1(x),self.ln1(x),self.ln1(x),
                             attn_mask=mask,
                             key_padding_mask=padding)
        x = x + att_out

        x = x + self.ffn(self.ln2(x))

        return x


In [23]:
class decoderNextWordPrediction(nn.Module):
    def __init__(self,vocab_size,d_model,block_size,n_heads,n_layers,dropRate):
        super().__init__()
        self.toke_embedding_table = nn.Embedding(vocab_size,d_model)
        pe = torch.zeros(block_size,d_model)
        position = torch.arange(0,block_size,dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer("pe",pe)#THIS IS NOT learnable parameter


        self.blocks = nn.ModuleList([Blocks(d_model,n_heads) for _ in range(n_layers)])

        self.ln_final = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model,vocab_size)

        self.register_buffer("mask",torch.triu(torch.ones(block_size,block_size),diagonal=1).bool())

    def forward(self,idx):#idx is the input word IDs of shape (B,T) T is the word it self
        B,T = idx.shape

        x = self.toke_embedding_table(idx) #(B,T,d_model)

        x = x + self.pe[:T,:] #you slice beacuse PE was created for the maximum sentance length

        current_mask = self.mask[:T,:T]

        for block in self.blocks:
            x = block(x,mask=current_mask)

        x = self.ln_final(x)
        logits = self.lm_head(x)


        return logits

In [24]:
configuration_hyperParameters = {
    "batch_size": 32,
    "learning_rate":3e-4,
    "epochs":5,
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

configuration_Model = {
    "block_size":None,
    "layers":4,
    "heads":4,
    "d_model":256,
    "vocab_size":None,
    "dropRate":0.1
}

